<a href="https://colab.research.google.com/github/weizetan/3179/blob/main/Balance/Balance_MMnet_casme2_4dme_samm_smic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyunpack patool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 3.5 MB/s eta 0:00:00


# **CASME2**

## **Check path exists**

In [ ]:
import os

def create_folder(path):
    """Create a folder and log progress."""
    if not os.path.exists(path):
        try:
            os.makedirs(path)
            print(f"Folder created: {path}")
        except Exception as e:
            print(f"Error creating folder {path}: {e}")
    else:
        print(f"Folder already exists: {path}")

# Example usage
path_to_create = '/content/drive/MyDrive/FYP/dataset/CASME2/sub1'  # Adjust path if needed
create_folder(path_to_create)


Folder already exists: /content/drive/MyDrive/FYP/dataset/CASME2/sub1


## **Unzip dataset**

In [ ]:
# from pyunpack import Archive
# import os

# def extract_rar(rar_file_path, extract_to_path):
#     """Extracts a rar file to a specified directory."""
#     Archive(rar_file_path).extractall(extract_to_path)
#     print(f"Extraction completed. Files are available in {extract_to_path}")

# # Define paths
# rar_file_path = './drive/MyDrive/FYP/dataset/CASME2.zip'  # Adjust this path if needed
# extract_to_path = './drive/MyDrive/FYP/dataset/'  # Adjust this path if needed

# # Make sure the extraction directory exists
# os.makedirs(extract_to_path, exist_ok=True)

# # Perform extraction
# extract_rar(rar_file_path, extract_to_path)

Extraction completed. Files are available in ./drive/MyDrive/FYP/


In [ ]:
import pandas as pd

# Load the Excel file
excel_file = './drive/MyDrive/FYP/dataset/CASME2/CASME2-coding-20140508.xlsx'
df_excel = pd.read_excel(excel_file)

# Inspect the first few rows
print(df_excel.head())

   Subject  Filename  Unnamed: 2  OnsetFrame  ApexFrame  OffsetFrame  \
0        1  EP02_01f         NaN          46         59           86   
1        1   EP03_02         NaN         131        139          161   
2        1   EP04_02         NaN          21         54           76   
3        1   EP04_03         NaN          31         41           56   
4        1   EP04_04         NaN          23         49           66   

   Unnamed: 6 Action Units Estimated Emotion (7 class)  Unnamed: 9  \
0         NaN           12                   happiness         NaN   
1         NaN           18                      others         NaN   
2         NaN            4                      others         NaN   
3         NaN            4                      others         NaN   
4         NaN            4                      others         NaN   

   Unnamed: 10  Unnamed: 11     Unnamed: 12  
0          NaN            1  sub01_EP02_01f  
1          NaN            2   sub01_EP03_02  
2       

## **Count images**

In [ ]:
# Count occurrences of each emotion
emotion_counts = df_excel['Estimated Emotion (7 class)'].value_counts()

# Print the counts
print("Emotion Counts:")
print(emotion_counts)

Emotion Counts:
Estimated Emotion (7 class)
others        99
disgust       63
happiness     32
repression    27
surprise      25
sadness        7
fear           2
Name: count, dtype: int64


In [ ]:
# prompt: remove 'others'.

# Filter out 'others' emotion
df_filtered = df_excel[df_excel['Estimated Emotion (7 class)'] != 'others']
df_filtered

# Count occurrences of each emotion
emotion_counts = df_filtered['Estimated Emotion (7 class)'].value_counts()

# Replace specific emotions with 'others'
emotions_to_replace = ['disgust', 'repression', 'sadness', 'fear']
df_filtered['Estimated Emotion (7 class)'] = df_filtered['Estimated Emotion (7 class)'].replace(emotions_to_replace, 'others')

# Count occurrences of each emotion after replacement
emotion_counts = df_filtered['Estimated Emotion (7 class)'].value_counts()

# Print the counts
print("Emotion Counts after replacing with 'others':")
print(emotion_counts)

Emotion Counts after replacing with 'others':
Estimated Emotion (7 class)
others       99
happiness    32
surprise     25
Name: count, dtype: int64


<ipython-input-5-0a62328e8f6a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Estimated Emotion (7 class)'] = df_filtered['Estimated Emotion (7 class)'].replace(emotions_to_replace, 'others')


In [ ]:
df_excel.shape[0]

255

In [ ]:
df_filtered = df_filtered[['Subject', 'Filename', 'Estimated Emotion (7 class)']]
df_filtered.columns = ['Subject', 'Filename', 'Emotion']
df_filtered

,Subject,Filename,Emotion
0,1,EP02_01f,happiness
7,1,EP19_05f,others
8,1,EP19_06f,others
9,2,EP01_11f,others
10,2,EP02_04f,others
...,...,...,...
249,26,EP18_44,others
251,26,EP18_47,others
252,26,EP18_49,others
253,26,EP18_50,others


In [ ]:
df_filtered.shape[0]

156

In [ ]:
df_filtered

,Subject,Filename,Emotion
0,1,EP02_01f,happiness
7,1,EP19_05f,others
8,1,EP19_06f,others
9,2,EP01_11f,others
10,2,EP02_04f,others
...,...,...,...
249,26,EP18_44,others
251,26,EP18_47,others
252,26,EP18_49,others
253,26,EP18_50,others


In [ ]:
import os
import pandas as pd

def count_images_by_emotion(base_dir, df_filtered):
    # Create a dictionary to store counts
    emotion_counts = {}

    # Filter DataFrame to only include relevant columns
    df_filtered = df_filtered[['Subject', 'Filename', 'Emotion']]
    df_filtered.columns = ['Subject', 'Filename', 'Emotion']

    # Iterate over each row in the DataFrame
    for _, row in df_filtered.iterrows():
        subject = f"sub{row['Subject']}"
        filename = row['Filename']
        emotion = row['Emotion']

        # Construct the directory path for the specific EP folder and emotion
        ep_folder_path = os.path.join(base_dir, subject, filename)

        if os.path.isdir(ep_folder_path):
            # Count images in this folder
            image_files = [f for f in os.listdir(ep_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            num_images = len(image_files)

            # Update the emotion count dictionary
            if emotion not in emotion_counts:
                emotion_counts[emotion] = 0
            emotion_counts[emotion] += num_images

    return emotion_counts

# Define base directory
base_dir = '/content/drive/MyDrive/FYP/dataset/CASME2/Cropped-updated/Cropped/'

# Load the filtered DataFrame (replace with your actual file path)
# Count images by emotion
emotion_counts = count_images_by_emotion(base_dir, df_filtered)

# Print the results
print("Total number of images per emotion:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count} images")


Total number of images per emotion:
happiness: 2360 images
others: 6792 images
surprise: 1605 images


## **Undersampling**

In [ ]:
import pandas as pd
import random

# Define the number of samples for 'others' to match 'happiness'
target_count = 32

# Separate out 'others' and other emotions
df_others = df_filtered[df_filtered['Emotion'] == 'others']
df_other_emotions = df_filtered[df_filtered['Emotion'] != 'others']

# Undersample 'others' to match the target count
if len(df_others) > target_count:
    df_others_sampled = df_others.sample(n=target_count, random_state=42)  # Ensure reproducibility
else:
    df_others_sampled = df_others  # If less than target_count, keep all

# Concatenate the sampled 'others' with the other emotions
df_balanced = pd.concat([df_other_emotions, df_others_sampled], ignore_index=True)

# Print the counts after undersampling
emotion_counts = df_balanced['Emotion'].value_counts()
print("Emotion Counts after undersampling 'others':")
print(emotion_counts)

# If you need to save the DataFrame to a CSV file
# df_balanced.to_csv('/path/to/your/undersampled_emotions.csv', index=False)


Emotion Counts after undersampling 'others':
Emotion
happiness    32
others       32
surprise     25
Name: count, dtype: int64


In [ ]:
df_balanced

,Subject,Filename,Emotion
0,1,EP02_01f,happiness
1,2,EP09_01,happiness
2,2,EP11_01,surprise
3,2,EP13_04,surprise
4,2,EP14_01,surprise
...,...,...,...
84,24,EP01_08,others
85,4,EP12_01f,others
86,19,EP13_01,others
87,26,EP18_44,others


### **copy u_train**

In [ ]:
import os
import shutil

# Define base and destination directories
base_dir = '/content/drive/MyDrive/FYP/dataset/CASME2/Cropped-updated/Cropped/'
dest_dir = '/content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/'

# Step 1: Copy files into emotion-based directories with 'u_train'
for _, row in df_balanced.iterrows():
    subject = f"sub{row['Subject']}"  # Use subject as 'sub1', 'sub2', etc.
    filename = row['Filename']
    emotion = row['Emotion']

    # Construct the source directory path
    subdir_path = os.path.join(base_dir, subject)

    # Iterate over files in the subject's directory to find matches
    for ep_dir in os.listdir(subdir_path):
        ep_dir_path = os.path.join(subdir_path, ep_dir)

        if os.path.isdir(ep_dir_path) and ep_dir == filename:
            # Create the destination EP directory under the emotion folder
            emotion_folder = 'others' if emotion == 'others' else emotion
            dest_ep_dir = os.path.join(dest_dir, subject, ep_dir)
            os.makedirs(dest_ep_dir, exist_ok=True)

            # Copy all image files within this EP directory
            for image_file in os.listdir(ep_dir_path):
                image_path = os.path.join(ep_dir_path, image_file)

                if os.path.isfile(image_path) and image_file.endswith(('.jpg', '.jpeg', '.png')):
                    dest_file = os.path.join(dest_ep_dir, image_file)

                    # Copy the file
                    shutil.copy(image_path, dest_file)
                    print(f"Copied {image_file} to {dest_file}")
                else:
                    print(f"Skipping {image_file} as it is not a recognized image file")

print("\nFile copying completed.")


Streaming output truncated to the last 5000 lines.
Copied reg_img52.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img52.jpg
Copied reg_img53.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img53.jpg
Copied reg_img54.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img54.jpg
Copied reg_img55.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img55.jpg
Copied reg_img56.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img56.jpg
Copied reg_img57.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img57.jpg
Copied reg_img58.jpg to /content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2_new/Cropped-updated/Cropped/sub9/EP09f/reg_img58.jpg
Copied reg_

### **Check**

In [ ]:
def count_ep_folders_by_emotion(base_dir):
  """Counts the number of EP folders for each emotion in the entire dataset."""
  emotion_counts = {}

  for subject in os.listdir(base_dir):
    subject_path = os.path.join(base_dir, subject)
    if os.path.isdir(subject_path):
      for folder in os.listdir(subject_path):
        folder_path = os.path.join(subject_path, folder)
        if os.path.isdir(folder_path):
          for emotion in os.listdir(folder_path):
            emotion_path = os.path.join(folder_path, emotion)
            if os.path.isdir(emotion_path):
              if emotion not in emotion_counts:
                emotion_counts[emotion] = 0
              emotion_counts[emotion] += len(os.listdir(emotion_path))

  return emotion_counts

# Count EP folders by emotion
base_dir = '/content/drive/MyDrive/FYP/dataset/mmnet_balanced_casme2/'
ep_counts = count_ep_folders_by_emotion(base_dir)

# Print the results
print("Total number of EP folders per emotion:")
for emotion, count in ep_counts.items():
  print(f"{emotion}: {count} EP folders")


Total number of EP folders per emotion:
happiness: 32 EP folders
others: 32 EP folders
surprise: 25 EP folders


In [ ]:
import os

def count_images(base_dir):
    """Counts the number of images in each emotion folder and prints debug information."""
    emotion_counts = {'happiness': 0, 'others': 0, 'surprise': 0}

    for subject in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject)
        if os.path.isdir(subject_path):
            train_path = os.path.join(subject_path, 'u_train')
            if os.path.isdir(train_path):
                # print(f"Processing train folder: {train_path}")  # Debug info
                for emotion_folder in os.listdir(train_path):
                    emotion_path = os.path.join(train_path, emotion_folder)
                    if os.path.isdir(emotion_path):
                        # print(f"Processing emotion folder: {emotion_path}")  # Debug info
                        for ep_folder in os.listdir(emotion_path):
                            ep_path = os.path.join(emotion_path, ep_folder)
                            if os.path.isdir(ep_path):
                                # print(f"Processing EP folder: {ep_path}")  # Debug info
                                image_files = [f for f in os.listdir(ep_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
                                # print(f"Found images: {image_files}")  # Debug info
                                num_images = len(image_files)
                                if emotion_folder in emotion_counts:
                                    emotion_counts[emotion_folder] += num_images
                                else:
                                    print(f"Unknown emotion folder: {emotion_folder}")
                            else:
                                print(f"Expected EP folder but found: {ep_path}")
            else:
                print(f"Expected 'u_train' folder but found: {train_path}")

    return emotion_counts

# Define the base directory
base_dir = '/content/drive/MyDrive/FYP/mmnet_balanced_casme22'

# Count images
emotion_counts = count_images(base_dir)

# Print the results
print("Total number of images per emotion:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count} images")


Total number of images per emotion:
happiness: 2360 images
others: 2175 images
surprise: 1605 images


In [ ]:
# prompt: duplicate '/content/drive/MyDrive/FYP/mmnet_balanced_casme22' this folder as '/content/drive/MyDrive/FYP/mmnet_balanced_casme22.2'

!cp -r '/content/drive/MyDrive/FYP/mmnet_balanced_casme22' '/content/drive/MyDrive/FYP/mmnet_balanced_casme22.2'


## **Copy u_test**

In [ ]:
import os
import shutil
import pandas as pd  # Assuming you use pandas for DataFrame handling

# Define base and destination directories
base_dir = '/content/drive/MyDrive/FYP/CASME2/Cropped-updated/Cropped/'
dest_dir = '/content/drive/MyDrive/FYP/mmnet_balanced_casme22/'

# Function to copy files into emotion-based directories for 'u_test'
def copy_files_to_emotion_dirs(df_filtered, base_dir, dest_dir, split_type):
    """
    Copy files from base_dir to emotion-based directories in dest_dir for the given split type ('u_test').
    """
    for _, row in df_filtered.iterrows():
        subject = f"sub{row['Subject']}"  # Use subject as 'sub1', 'sub2', etc.
        filename = row['Filename']
        emotion = row['Emotion']

        # Construct the source directory path
        subdir_path = os.path.join(base_dir, subject)

        # Iterate over files in the subject's directory to find matches
        for ep_dir in os.listdir(subdir_path):
            ep_dir_path = os.path.join(subdir_path, ep_dir)

            if os.path.isdir(ep_dir_path) and ep_dir == filename:
                # Create the destination EP directory under the emotion folder for 'u_test'
                emotion_folder = 'others' if emotion == 'others' else emotion
                dest_ep_dir = os.path.join(dest_dir, subject, 'u_test', emotion_folder, ep_dir)
                os.makedirs(dest_ep_dir, exist_ok=True)

                # Copy all image files within this EP directory
                for image_file in os.listdir(ep_dir_path):
                    image_path = os.path.join(ep_dir_path, image_file)

                    if os.path.isfile(image_path) and image_file.endswith(('.jpg', '.jpeg', '.png')):
                        dest_file = os.path.join(dest_ep_dir, image_file)

                        # Copy the file
                        shutil.copy(image_path, dest_file)
                        print(f"Copied {image_file} to {dest_file}")
                    else:
                        print(f"Skipping {image_file} as it is not a recognized image file")


# Step: Copy files into emotion-based directories for 'u_test'
copy_files_to_emotion_dirs(df_filtered, base_dir, dest_dir, 'u_test')

print("\nFile copying for u_test completed.")


Streaming output truncated to the last 5000 lines.
Copied reg_img179.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img179.jpg
Copied reg_img180.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img180.jpg
Copied reg_img181.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img181.jpg
Copied reg_img182.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img182.jpg
Copied reg_img183.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img183.jpg
Copied reg_img184.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img184.jpg
Copied reg_img185.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img185.jpg
Copied reg_img186.jpg to /content/drive/MyDrive/FYP/mmnet_balanced_casme22/sub17/u_test/others/EP02_11/reg_img186.jpg
Copie

### **Check**

In [ ]:
import os
import pandas as pd

def count_images_by_emotion(base_dir, df_filtered):
    # Create a dictionary to store counts
    emotion_counts = {}

    # Filter DataFrame to only include relevant columns
    df_filtered = df_filtered[['Subject', 'Filename', 'Emotion']]
    df_filtered.columns = ['Subject', 'Filename', 'Emotion']

    # Iterate over each row in the DataFrame
    for _, row in df_filtered.iterrows():
        subject = f"sub{row['Subject']}"
        filename = row['Filename']
        emotion = row['Emotion']

        # Construct the directory path for the specific EP folder and emotion
        ep_folder_path = os.path.join(base_dir, subject, filename)

        if os.path.isdir(ep_folder_path):
            # Count images in this folder
            image_files = [f for f in os.listdir(ep_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            num_images = len(image_files)

            # Update the emotion count dictionary
            if emotion not in emotion_counts:
                emotion_counts[emotion] = 0
            emotion_counts[emotion] += num_images

    return emotion_counts

# Define base directory
base_dir = '/content/drive/MyDrive/FYP/CASME2/Cropped-updated/Cropped/'

# Load the filtered DataFrame (replace with your actual file path)
# Count images by emotion
emotion_counts = count_images_by_emotion(base_dir, df_filtered)

# Print the results
print("Total number of images per emotion:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count} images")


Total number of images per emotion:
happiness: 2360 images
others: 6792 images
surprise: 1605 images


In [ ]:
import os

def count_ep_folders_by_emotion_in_u_test(base_dir):
    """Counts the number of EP folders for each emotion in the 'u_test' directory."""
    emotion_counts = {}

    # Iterate over each subject's directory
    for subject in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject)
        u_test_path = os.path.join(subject_path, 'u_test')  # Focus on 'u_test' directory

        if os.path.isdir(u_test_path):
            for emotion in os.listdir(u_test_path):
                emotion_path = os.path.join(u_test_path, emotion)
                if os.path.isdir(emotion_path):
                    if emotion not in emotion_counts:
                        emotion_counts[emotion] = 0
                    # Count EP directories under each emotion
                    for ep_folder in os.listdir(emotion_path):
                        ep_folder_path = os.path.join(emotion_path, ep_folder)
                        if os.path.isdir(ep_folder_path):
                            emotion_counts[emotion] += 1

    return emotion_counts

# Define base directory
base_dir = '/content/drive/MyDrive/FYP/mmnet_balanced_casme22/'

# Count EP folders in 'u_test' by emotion
ep_counts = count_ep_folders_by_emotion_in_u_test(base_dir)

# Print the results
print("Total number of EP folders in 'u_test' per emotion:")
for emotion, count in ep_counts.items():
    print(f"{emotion}: {count} EP folders")


Total number of EP folders in 'u_test' per emotion:
happiness: 32 EP folders
others: 99 EP folders
surprise: 25 EP folders


In [ ]:
import os

def count_images(base_dir):
    """Counts the number of images in each emotion folder and prints debug information."""
    emotion_counts = {'happiness': 0, 'others': 0, 'surprise': 0}

    for subject in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject)
        if os.path.isdir(subject_path):
            train_path = os.path.join(subject_path, 'u_test')
            if os.path.isdir(train_path):
                # print(f"Processing train folder: {train_path}")  # Debug info
                for emotion_folder in os.listdir(train_path):
                    emotion_path = os.path.join(train_path, emotion_folder)
                    if os.path.isdir(emotion_path):
                        # print(f"Processing emotion folder: {emotion_path}")  # Debug info
                        for ep_folder in os.listdir(emotion_path):
                            ep_path = os.path.join(emotion_path, ep_folder)
                            if os.path.isdir(ep_path):
                                # print(f"Processing EP folder: {ep_path}")  # Debug info
                                image_files = [f for f in os.listdir(ep_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
                                # print(f"Found images: {image_files}")  # Debug info
                                num_images = len(image_files)
                                if emotion_folder in emotion_counts:
                                    emotion_counts[emotion_folder] += num_images
                                else:
                                    print(f"Unknown emotion folder: {emotion_folder}")
                            else:
                                print(f"Expected EP folder but found: {ep_path}")
            else:
                print(f"Expected 'u_train' folder but found: {train_path}")

    return emotion_counts

# Define the base directory
base_dir = '/content/drive/MyDrive/FYP/mmnet_balanced_casme22'

# Count images
emotion_counts = count_images(base_dir)

# Print the results
print("Total number of images per emotion:")
for emotion, count in emotion_counts.items():
    print(f"{emotion}: {count} images")


Total number of images per emotion:
happiness: 2360 images
others: 6792 images
surprise: 1605 images


## **Compare**

In [ ]:
df_filtered


,Subject,Filename,Emotion
0,1,EP02_01f,happiness
7,1,EP19_05f,others
8,1,EP19_06f,others
9,2,EP01_11f,others
10,2,EP02_04f,others
...,...,...,...
249,26,EP18_44,others
251,26,EP18_47,others
252,26,EP18_49,others
253,26,EP18_50,others


In [ ]:
# prompt: can i get the rows which not in df_balanced

# Find rows in df_filtered that are not in df_balanced
df_not_in_balanced = df_filtered[~df_filtered.apply(tuple, 1).isin(df_balanced.apply(tuple, 1))]

# Print the result
df_not_in_balanced


,Subject,Filename,Emotion
8,1,EP19_06f,others
9,2,EP01_11f,others
10,2,EP02_04f,others
12,2,EP06_01f,others
13,2,EP06_02f,others
...,...,...,...
246,26,EP13_11,others
247,26,EP15_01,others
248,26,EP16_01,others
252,26,EP18_49,others


# **4DME**

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = './drive/MyDrive/FYP/dataset/4DME/combined_predictions_and_labels.xlsx'
df_excel = pd.read_excel(excel_file)

In [ ]:
# Inspect the first few rows
df_excel

,filename,Predicted Label,Actual Label
0,S08_003_02_2,1,2
1,S08_005_01_1,1,2
2,S08_009_01_1,1,2
3,S08_003_02_2,2,2
4,S08_005_01_1,2,2
...,...,...,...
12000,S65_003_06_1,2,1
12001,S65_003_06_2,2,1
12002,S65_003_06_3,2,1
12003,S65_007_01_1,2,2


In [ ]:
df_excel = df_excel.drop(columns=['Predicted Label'])

In [ ]:
import os
import shutil
import pandas as pd

# Define source and destination directories
source_dir = './drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/'
dest_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/'

# Step 1: Check valid filenames
valid_rows = []
for index, row in df_excel.iterrows():
    filename = row['filename']

    # Extract subject based on filename length
    parts = filename.split("_")
    if len(filename) == 14:  # For filenames like S58_1_007_01_1
        subject = parts[0] + "_" + parts[1]
    elif len(filename) == 12:  # For filenames like S08_003_02_2
        subject = parts[0]
    else:
        print(f"Filename {filename} does not match expected format.")
        continue

    image_folder_path = os.path.join(source_dir, subject, filename)
    if os.path.exists(image_folder_path):
        valid_rows.append(row)
    else:
        print(f"File {filename} not found. Skipping this row.")

df_valid = pd.DataFrame(valid_rows)


In [ ]:
# Remove duplicates if any
df_no_duplicates = df_valid.drop_duplicates()

## **Check valid filename**

In [ ]:
# Step 2: Perform undersampling
emotion_counts = df_no_duplicates['Actual Label'].value_counts()
min_count = emotion_counts.min()

df_undersampled = pd.DataFrame(columns=['filename', 'Actual Label'])

for emotion in emotion_counts.index:
    df_emotion = df_no_duplicates[df_no_duplicates['Actual Label'] == emotion]
    if len(df_emotion) >= min_count:
        df_sampled = df_emotion.sample(n=min_count, random_state=42)
    else:
        df_sampled = df_emotion  # If not enough samples, take all available samples
    df_undersampled = pd.concat([df_undersampled, df_sampled], ignore_index=True)

print("Emotion Counts in Undersampled DataFrame:")
print(df_undersampled['Actual Label'].value_counts())


Emotion Counts in Undersampled DataFrame:
Actual Label
2    25
0    25
1    25
Name: count, dtype: int64


## **Count images**

In [ ]:
# Count imges from ori files
import os
import pandas as pd

# Load the Excel file
excel_file = './drive/MyDrive/FYP/dataset/4DME/combined_predictions_and_labels.xlsx'
df_excel = pd.read_excel(excel_file)

# Define the base source directory
base_source_dir = './drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/'

def count_images_in_all_subjects(base_directory):
    # Initialize a dictionary to store image counts for each subject
    image_counts = {}

    # Iterate through all directories in the base directory
    for subject in os.listdir(base_directory):
        subject_path = os.path.join(base_directory, subject)
        if os.path.isdir(subject_path):
            # Initialize count for the current subject
            total_images = 0
            for filename in os.listdir(subject_path):
                filename_path = os.path.join(subject_path, filename)
                if os.path.isdir(filename_path):
                    # Count the number of .jpg files in the current filename directory
                    num_images = len([f for f in os.listdir(filename_path) if f.endswith('.jpg')])
                    total_images += num_images
            # Store the count for the current subject
            image_counts[subject] = total_images

    return image_counts

# Get the image counts
image_counts = count_images_in_all_subjects(base_source_dir)

# Map subjects to labels, normalize subject names for comparison
subject_to_label = {}
for index, row in df_excel.iterrows():
    subject_name = row['filename'].split('_')[0]  # Adjust this to fit your naming pattern
    label = row['Actual Label']
    subject_to_label[subject_name] = label

# Initialize a dictionary to store image counts by label
label_image_counts = {0: 0, 1: 0, 2: 0}

# Aggregate image counts by label
for subject, count in image_counts.items():
    # Normalize subject names for comparison
    subject_key = subject.split('_')[0]  # Adjust this to fit your naming pattern
    label = subject_to_label.get(subject_key, None)
    if label is not None:
        label_image_counts[label] += count

# Print the results
print("Image Counts by Label:")
for label, count in label_image_counts.items():
    print(f"Label {label}: {count} images")


Image Counts by Label:
Label 0: 1207 images
Label 1: 945 images
Label 2: 3563 images


## **Undersampling**

In [ ]:
df = df_undersampled

In [ ]:
import os

def count_folders_by_label(base_dir):
    """Counts the number of folders for each label (0, 1, 2) in the u_train directories."""
    label_counts = {0: 0, 1: 0, 2: 0}

    # Iterate through all subjects in the base directory
    for subject in os.listdir(base_dir):
        subject_path = os.path.join(base_dir, subject)

        # Ensure the subject directory exists and is a directory
        if os.path.isdir(subject_path):
            train_path = os.path.join(subject_path, 'u_train')

            # Check if the u_train directory exists for the subject
            if os.path.isdir(train_path):
                # Iterate through each label (0, 1, 2)
                for label in label_counts.keys():
                    label_folder_path = os.path.join(train_path, str(label))

                    # Count the number of folders in the label directory
                    if os.path.isdir(label_folder_path):
                        label_counts[label] += len(os.listdir(label_folder_path))

    return label_counts

# Define the base directory (where subject folders are located)
base_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/'

# Count folders for each label
label_folder_counts = count_folders_by_label(base_dir)

# Print the results
print("Number of folders for each Actual Label in 'u_train':")
for label, count in label_folder_counts.items():
    print(f"Actual Label {label}: {count} folders")


Number of folders for each Actual Label in 'u_train':
Actual Label 0: 25 folders
Actual Label 1: 25 folders
Actual Label 2: 25 folders


In [ ]:
import os
import shutil
import pandas as pd

# Define source and destination directories
source_dir = './drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/'
dest_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray_video/'

# Initialize counters for each label
copy_counts = {0: 0, 1: 0, 2: 0}

# Iterate through the undersampled DataFrame
for index, row in df_undersampled.iterrows():
    filename = row['filename']
    label = row['Actual Label']

    # Extract subject based on filename length
    parts = filename.split("_")
    if len(filename) == 14:  # For filenames like S58_1_007_01_1
        subject = parts[0] + "_" + parts[1]  # E.g., S58_1
    elif len(filename) == 12:  # For filenames like S08_003_02_2
        subject = parts[0]  # E.g., S08
    else:
        print(f"Filename {filename} does not match expected format.")
        continue

    # Create the destination directory path
    dest_ep_dir = os.path.join(dest_dir, subject, filename)
    os.makedirs(dest_ep_dir, exist_ok=True)

    # Source folder path (assuming the subject directory contains multiple image files)
    source_folder_path = os.path.join(source_dir, subject, filename)  # Adjust if the structure is different

    # Check if the source folder exists
    if os.path.exists(source_folder_path):
        # Copy images from the source folder to the destination
        for image_file in os.listdir(source_folder_path):
            if image_file.endswith(".jpg"):  # Check if the file is a JPG image
                src_image_path = os.path.join(source_folder_path, image_file)
                dest_image_path = os.path.join(dest_ep_dir, image_file)

                # Check if the source image exists before copying
                if os.path.exists(src_image_path):
                    shutil.copy(src_image_path, dest_image_path)  # Copy the image
                    copy_counts[label] += 1  # Count successful copy
                    print(f"Copied {src_image_path} to {dest_image_path}")
                else:
                    print(f"Source image not found: {src_image_path}")
    else:
        print(f"Source folder not found: {source_folder_path}")

# Print the number of copied files for each label
print("\nFile copying summary:")
for label, count in copy_counts.items():
    print(f"Label {label}: {count} files copied")


Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006011.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006011.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006014.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006014.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006013.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006013.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S63/S63_002_01_3/Frame_000006016.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray

In [ ]:
def count_folders_in_sub(base_dir):
  """Counts the number of folders within each subject's 'u_train' directory."""
  subject_folder_counts = {}

  # Iterate through all subjects in the base directory
  for subject in os.listdir(base_dir):
    subject_path = os.path.join(base_dir, subject)

    # Ensure the subject directory exists and is a directory
    if os.path.isdir(subject_path):
      train_path = os.path.join(subject_path)

      # Check if the u_train directory exists for the subject
      if os.path.isdir(train_path):
        # Count the number of folders within the 'u_train' directory
        folder_count = len([f for f in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, f))])
        subject_folder_counts[subject] = folder_count

  return subject_folder_counts

# Define the base directory (where subject folders are located)
base_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_4DME_new/micro_short_gray_video/micro_short_gray_video/'

# Count folders for each subject
subject_folder_counts = count_folders_in_sub(base_dir)

# Print the results
print("Number of folders in 'u_train' for each subject:")
for subject, count in subject_folder_counts.items():
    print(f"{subject}: {count} folders")


Number of folders in 'u_train' for each subject:
S63: 2 folders
S34: 1 folders
S18: 1 folders
S56: 2 folders
S22: 1 folders
S12: 2 folders
S25: 1 folders
S37: 1 folders
S60: 5 folders
S30_1: 2 folders
S53: 7 folders
S42: 4 folders
S16: 3 folders
S49: 1 folders
S38: 1 folders
S20: 1 folders
S55: 1 folders
S09: 1 folders
S57: 2 folders
S52_2: 1 folders
S41: 1 folders
S27: 3 folders
S61: 4 folders
S51: 1 folders
S46: 1 folders
S13_2: 1 folders
S50_1: 6 folders
S65: 8 folders
S14: 4 folders
S30_2: 2 folders
S10: 2 folders
S13_1: 1 folders
S58_1: 1 folders


## **Copy u_test**

In [ ]:
import os
import shutil
import pandas as pd

# Define source and destination directories
source_dir = './drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/'
dest_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/'

# Iterate through the undersampled DataFrame
for index, row in df_excel.iterrows():
    filename = row['filename']
    label = row['Actual Label']

    # Extract subject based on filename length
    parts = filename.split("_")
    if len(filename) == 14:  # For filenames like S58_1_007_01_1
        subject = parts[0] + "_" + parts[1]  # E.g., S58_1
    elif len(filename) == 12:  # For filenames like S08_003_02_2
        subject = parts[0]  # E.g., S08
    else:
        print(f"Filename {filename} does not match expected format.")
        continue

    # Create the destination directory path
    dest_ep_dir = os.path.join(dest_dir, subject, 'u_test', str(label), filename)
    os.makedirs(dest_ep_dir, exist_ok=True)

    # Source folder path (assuming the subject directory contains multiple image files)
    source_folder_path = os.path.join(source_dir, subject, filename)  # Adjust if the structure is different

    # Check if the source folder exists
    if os.path.exists(source_folder_path):
        # Destination path for images
        for image_file in os.listdir(source_folder_path):
            if image_file.endswith(".jpg"):  # Check if the file is a JPG image
                src_image_path = os.path.join(source_folder_path, image_file)
                dest_image_path = os.path.join(dest_ep_dir, image_file)

                # Check if the source image exists before copying
                if os.path.exists(src_image_path):
                    shutil.copy(src_image_path, dest_image_path)  # Copy the image
                    print(f"Copied {src_image_path} to {dest_image_path}")
                else:
                    print(f"Source image not found: {src_image_path}")
    else:
        print(f"Source folder not found: {source_folder_path}")

Streaming output truncated to the last 5000 lines.
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S65/S65_003_06_3/Frame_000006938.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/S65/u_test/1/S65_003_06_3/Frame_000006938.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S65/S65_003_06_3/Frame_000006934.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/S65/u_test/1/S65_003_06_3/Frame_000006934.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S65/S65_003_06_3/Frame_000006933.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/S65/u_test/1/S65_003_06_3/Frame_000006933.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_short_gray_video/S65/S65_003_06_3/Frame_000006937.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_4DME/S65/u_test/1/S65_003_06_3/Frame_000006937.jpg
Copied ./drive/MyDrive/FYP/dataset/4DME/micro_short_gray_video/micro_

# **SAMM**

In [ ]:
!pip install openpyxl

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/FYP/dataset/SAMM/SAMM_Micro_FACS_Codes_v2.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')  # Use openpyxl engine for xlsx files

print(df.head())  # Display the first few rows of the DataFrame


   Subject Filename  Inducement Code  Onset Frame  Apex Frame  Offset Frame  \
0        6  006_1_2                1         5562        5588          5632   
1        6  006_1_3                1         3912        3948          3988   
2        6  006_1_4                1         2324        2368          2403   
3        6  006_1_5                1         5343        5388          5424   
4        6  006_1_6                1         7160        7197          7259   

   Duration        Micro Action Units Estimated Emotion  Objective Classes  \
0        71  Micro - 1/2          4+7             anger                  3   
1        77  Micro - 1/2            4             anger                  3   
2        80  Micro - 1/2          4+7             anger                  3   
3        82  Micro - 1/2          4+7             anger                  3   
4       100  Micro - 1/2       4+7+43             anger                  7   

            Notes  
0             NaN  
1  While blinkin

In [ ]:
df.shape[0]

159

In [ ]:
emotion_counts = df['Estimated Emotion'].value_counts()
print(emotion_counts)

Estimated Emotion
anger        57
other        26
happiness    26
surprise     15
contempt     12
disgust       9
fear          8
sadness       6
Name: count, dtype: int64


In [ ]:
# Create a dictionary to map emotions to categories
emotion_mapping = {
    'anger': 'negative',
    'contempt': 'negative',
    'disgust': 'negative',
    'fear': 'negative',
    'sadness': 'negative',
    'happiness': 'positive',
    'surprise': 'surprise'
}

# Add a new column 'Emotion Category' to the DataFrame
df['Emotion Category'] = df['Estimated Emotion'].map(emotion_mapping)

# Display the updated DataFrame to check the new column
print(df.head())

# Count the occurrences of each emotion category
emotion_category_counts = df['Emotion Category'].value_counts()
print(emotion_category_counts)


   Subject Filename  Inducement Code  Onset Frame  Apex Frame  Offset Frame  \
0        6  006_1_2                1         5562        5588          5632   
1        6  006_1_3                1         3912        3948          3988   
2        6  006_1_4                1         2324        2368          2403   
3        6  006_1_5                1         5343        5388          5424   
4        6  006_1_6                1         7160        7197          7259   

   Duration        Micro Action Units Estimated Emotion  Objective Classes  \
0        71  Micro - 1/2          4+7             anger                  3   
1        77  Micro - 1/2            4             anger                  3   
2        80  Micro - 1/2          4+7             anger                  3   
3        82  Micro - 1/2          4+7             anger                  3   
4       100  Micro - 1/2       4+7+43             anger                  7   

            Notes Emotion Category  
0             NaN  

In [ ]:
# Create a new DataFrame without rows where 'Estimated Emotion' is 'others'
df_no_others = df[df['Estimated Emotion'] != 'other']

# Print the updated DataFrame to check the changes
print(df_no_others.head())

# Alternatively, you can modify the existing DataFrame:
# df = df[df['Estimated Emotion'] != 'others']


   Subject Filename  Inducement Code  Onset Frame  Apex Frame  Offset Frame  \
0        6  006_1_2                1         5562        5588          5632   
1        6  006_1_3                1         3912        3948          3988   
2        6  006_1_4                1         2324        2368          2403   
3        6  006_1_5                1         5343        5388          5424   
4        6  006_1_6                1         7160        7197          7259   

   Duration        Micro Action Units Estimated Emotion  Objective Classes  \
0        71  Micro - 1/2          4+7             anger                  3   
1        77  Micro - 1/2            4             anger                  3   
2        80  Micro - 1/2          4+7             anger                  3   
3        82  Micro - 1/2          4+7             anger                  3   
4       100  Micro - 1/2       4+7+43             anger                  7   

            Notes Emotion Category  
0             NaN  

In [ ]:
df_no_others.shape[0]

133

In [ ]:
df_filtered = df_no_others

In [ ]:
# Assuming 'df_filtered' is your DataFrame and you want to downsample 'negative' to 50

# Filter rows with 'negative' emotion
df_negative = df_filtered[df_filtered['Emotion Category'] == 'negative']

# Check if there are more than 50 'negative' samples
if len(df_negative) > 30:
  # Downsample to 50 using random sampling
  df_negative_downsampled = df_negative.sample(n=30, random_state=42)  # Use random_state for reproducibility

  # Remove the original 'negative' samples from the DataFrame
  df_filtered = df_filtered[df_filtered['Emotion Category'] != 'negative']

  # Concatenate the downsampled 'negative' samples back to the DataFrame
  df_filtered = pd.concat([df_filtered, df_negative_downsampled], ignore_index=True)

# Now df_filtered contains the downsampled data with 'negative' emotion count at most 50
print("Emotion Counts in Downsampled DataFrame:")
print(df_filtered['Emotion Category'].value_counts())



Emotion Counts in Downsampled DataFrame:
Emotion Category
negative    30
positive    26
surprise    15
Name: count, dtype: int64


In [ ]:
df_undersampled = df_filtered

In [ ]:
df_undersampled.head()

,Subject,Filename,Inducement Code,Onset Frame,Apex Frame,Offset Frame,Duration,Micro,Action Units,Estimated Emotion,Objective Classes,Notes,Emotion Category
0,6,006_3_5,3,1787,1829,1856,70,Micro - 1/2,25+26,surprise,2,NaN,surprise
1,6,006_5_9,5,5258,5282,5351,94,Micro - 1/2,5,surprise,2,NaN,surprise
2,7,007_5_3,5,2759,2797,2835,77,Micro - 1/2,26,surprise,7,NaN,surprise
3,7,007_6_1,6,497,511,541,45,Micro - 1/2,6+12+15,happiness,7,NaN,positive
4,7,007_6_2,6,1296,1326,1377,82,Micro - 1/2,12,happiness,1,NaN,positive


In [ ]:
df_undersampled['Subject'] = df_undersampled['Subject'].astype(str).str.zfill(3)

print(df_undersampled['Subject'])


0     006
1     006
2     007
3     007
4     007
     ... 
66    009
67    026
68    030
69    012
70    033
Name: Subject, Length: 71, dtype: object


In [ ]:
print(df_undersampled['Emotion Category'].value_counts())

Emotion Category
negative    30
positive    26
surprise    15
Name: count, dtype: int64


In [ ]:
# prompt: save as excel

# Assuming df_undersampled is the DataFrame you want to save
df_undersampled.to_excel('/content/drive/MyDrive/FYP/dataset/balanced_SAMM.xlsx', index=False)

In [ ]:
import os
import shutil
import pandas as pd

# Define source and destination directories
source_dir = './drive/MyDrive/FYP/dataset/SAMM/'
dest_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/'

# Initialize counters for each label
copy_counts = {'surprise': 0, 'negative': 0, 'positive': 0}

# Iterate through the undersampled DataFrame
for index, row in df_undersampled.iterrows():
    subject = row['Subject']
    filename = row['Filename']
    label = row['Emotion Category']

    # Create the destination directory path
    dest_ep_dir = os.path.join(dest_dir, subject, filename)

    # Ensure all parent directories in dest_ep_dir exist
    os.makedirs(dest_ep_dir, exist_ok=True)

    # Source folder path (assuming the subject directory contains multiple image files)
    source_folder_path = os.path.join(source_dir, subject, filename)  # Adjust if the structure is different

    # Check if the source folder exists
    if os.path.exists(source_folder_path):
        # Copy images from the source folder to the destination
        for image_file in os.listdir(source_folder_path):
            if image_file.endswith(".jpg"):  # Check if the file is a JPG image
                src_image_path = os.path.join(source_folder_path, image_file)
                dest_image_path = os.path.join(dest_ep_dir, image_file)

                # Check if the source image exists before copying
                if os.path.exists(src_image_path):
                    shutil.copy(src_image_path, dest_image_path)  # Copy the image
                    copy_counts[label] += 1  # Count successful copy
                    print(f"Copied {src_image_path} to {dest_image_path}")
                else:
                    print(f"Source image not found: {src_image_path}")
    else:
        print(f"Source folder not found: {source_folder_path}")

# Print the number of copied files for each label
print("\nFile copying summary:")
for label, count in copy_counts.items():
    print(f"Label {label}: {count} files copied")

Streaming output truncated to the last 5000 lines.
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5289.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5289.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5320.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5320.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5313.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5313.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5272.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5272.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5293.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5293.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5333.jpg to ./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/006/006_5_9/006_5333.jpg
Copied ./drive/MyDrive/FYP/dataset/SAMM/006/006_5_9/006_5259.jpg to ./drive/MyDrive/FYP/dataset/mmnet_b

In [ ]:
# Initialize a dictionary to store the counts for each emotion category
emotion_file_counts = {'surprise': 0, 'negative': 0, 'positive': 0}

# Iterate through the df_undersampled DataFrame
for index, row in df_undersampled.iterrows():
  emotion_category = row['Emotion Category']
  subject = row['Subject']
  filename = row['Filename']

  # Construct the full file path
  file_path = os.path.join('./drive/MyDrive/FYP/dataset/mmnet_balanced_SAMM/', subject, filename)

  # Check if the file path exists and increment the corresponding count
  if os.path.exists(file_path):
    emotion_file_counts[emotion_category] += 1

# Print the results
print("Number of files for each emotion category:")
for emotion, count in emotion_file_counts.items():
  print(f"{emotion}: {count}")


Number of files for each emotion category:
surprise: 15
negative: 30
positive: 26


# **SMIC**

In [ ]:
!wget "https://monashuni-my.sharepoint.com/:f:/g/personal/tan_peisze_monash_edu/EijLg93Ny79Fpbby81N4eSgBSqx-zXj3WCatx7Vh563Zqw?download=1" -O /content/drive/MyDrive/FYP/dataset/SMIC.zip


--2024-10-03 10:45:16--  https://monashuni-my.sharepoint.com/:f:/g/personal/tan_peisze_monash_edu/EijLg93Ny79Fpbby81N4eSgBSqx-zXj3WCatx7Vh563Zqw?download=1
Resolving monashuni-my.sharepoint.com (monashuni-my.sharepoint.com)... 13.107.136.10, 13.107.138.10, 2620:1ec:8f8::10, ...
Connecting to monashuni-my.sharepoint.com (monashuni-my.sharepoint.com)|13.107.136.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/tan_peisze_monash_edu/Documents/SMIC?ga=1 [following]
--2024-10-03 10:45:17--  https://monashuni-my.sharepoint.com/personal/tan_peisze_monash_edu/Documents/SMIC?ga=1
Reusing existing connection to monashuni-my.sharepoint.com:443.
HTTP request sent, awaiting response... 302 Found
Location: /personal/tan_peisze_monash_edu/Documents/Forms/All.aspx?RootFolder=%2fpersonal%2ftan%5fpeisze%5fmonash%5fedu%2fDocuments%2fSMIC&FolderCTID=0x01200014F00F61CC47FF49AB14F11F5ADE57D5&ga=1 [following]
--2024-10-03 10:45:17--  https://monashuni-my.sharepoint.c

In [ ]:
!ls -lh /content/drive/MyDrive/FYP/dataset/SMIC.zip


-rw------- 1 root root 292K Oct  3 10:45 /content/drive/MyDrive/FYP/dataset/SMIC.zip


In [ ]:
!file /content/drive/MyDrive/FYP/dataset/SMIC.zip


/content/drive/MyDrive/FYP/dataset/SMIC.zip: HTML document, ASCII text, with very long lines (65047), with CRLF line terminators


In [ ]:
!unzip "/content/drive/MyDrive/FYP/dataset/SMIC.zip" -d "/content/drive/MyDrive/FYP/dataset/"


Archive:  /content/drive/MyDrive/FYP/dataset/SMIC.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/FYP/dataset/SMIC.zip or
        /content/drive/MyDrive/FYP/dataset/SMIC.zip.zip, and cannot find /content/drive/MyDrive/FYP/dataset/SMIC.zip.ZIP, period.


In [ ]:
# prompt: how to read excel file

import pandas as pd

# Replace 'your_excel_file.xlsx' with the actual path to your Excel file
excel_file_path = './drive/MyDrive/FYP/dataset/SAMM/SMIC-HS-E_annotation.xlsx'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file_path)

# Now you can work with the DataFrame 'df'
print(df.columns)  # Print the first few rows of the DataFrame


Index(['Subject', 'Filename', 'Unnamed: 2', 'OnsetF', 'OffsetF', 'Onset2F',
       'Offset2F', 'Onset3F', 'Offset3F', 'Unnamed: 9', 'FirstF', 'LastF',
       'TotalMF1', 'TotalMF2', 'TotalMF3', 'TotalVL', 'Emotion',
       'Emotion number ', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21'],
      dtype='object')


In [ ]:
emotion_counts = df['Emotion number '].value_counts()
print(emotion_counts)

Emotion number 
0    64
1    51
2    42
Name: count, dtype: int64


In [ ]:
# Assuming 'df_filtered' is your DataFrame and you want to downsample 'negative' to 50

# Filter rows with 'negative' emotion
df_negative = df[df['Emotion number '] == 0]

# Check if there are more than 51 'negative' samples
if len(df_negative) > 51:
  # Downsample to 50 using random sampling
  df_negative_downsampled = df_negative.sample(n=51, random_state=42)  # Use random_state for reproducibility

  # Remove the original 'negative' samples from the DataFrame
  df = df[df['Emotion number '] != 0]

  # Concatenate the downsampled 'negative' samples back to the DataFrame
  df = pd.concat([df, df_negative_downsampled], ignore_index=True)

# Now df_filtered contains the downsampled data with 'negative' emotion count at most 50
print("Emotion Counts in Downsampled DataFrame:")
print(df['Emotion number '].value_counts())

Emotion Counts in Downsampled DataFrame:
Emotion number 
1    51
0    51
2    42
Name: count, dtype: int64


In [ ]:
df

,Subject,Filename,Unnamed: 2,OnsetF,OffsetF,Onset2F,Offset2F,Onset3F,Offset3F,Unnamed: 9,...,TotalMF1,TotalMF2,TotalMF3,TotalVL,Emotion,Emotion number,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
0,20,s20_po_01,NaN,17598752,17598798,NaN,NaN,NaN,NaN,NaN,...,47,NaN,NaN,212,positive,1,NaN,NaN,1,s20_po_01
1,20,s20_po_02,NaN,17646868,17646910,NaN,NaN,NaN,NaN,NaN,...,43,NaN,NaN,800,positive,1,NaN,NaN,1,s20_po_02
2,20,s20_po_03,NaN,17748546,17748570,NaN,NaN,NaN,NaN,NaN,...,25,NaN,NaN,280,positive,1,NaN,NaN,1,s20_po_03
3,20,s20_sur_01,NaN,17597115,17597152,NaN,NaN,NaN,NaN,NaN,...,38,NaN,NaN,720,surprise,2,NaN,NaN,2,s20_sur_01
4,20,s20_sur_02,NaN,17692849,17692870,17693037.0,17693057.0,17549981.0,17692781.0,NaN,...,22,20.0,-143076.0,400,surprise,2,NaN,NaN,2,s20_sur_02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,11,s11_ne_02,NaN,514569,514599,NaN,NaN,NaN,NaN,NaN,...,31,NaN,NaN,400,negative,0,NaN,NaN,0,s11_ne_02
140,20,s20_ne_03,NaN,17625391,17625424,NaN,NaN,NaN,NaN,NaN,...,34,NaN,NaN,200,negative,0,NaN,NaN,0,s20_ne_03
141,8,s8_ne_08,NaN,399100,399127,NaN,NaN,NaN,NaN,NaN,...,28,NaN,NaN,500,negative,0,NaN,NaN,0,s8_ne_08
142,3,s3_ne_17,NaN,425101,425134,NaN,NaN,NaN,NaN,NaN,...,34,NaN,NaN,800,negative,0,NaN,NaN,0,s3_ne_17


In [ ]:
# prompt: save df to excel file

# Specify the path and filename for the Excel file
excel_file_path = '/content/drive/MyDrive/FYP/dataset/balanced_SMIC-HS-E_annotation.xlsx'

# Save the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to: {excel_file_path}")


DataFrame saved to: /content/drive/MyDrive/FYP/dataset/balanced_SMIC-HS-E_annotation.xlsx


In [ ]:
import zipfile
import os
import shutil

# Assuming 'df' is your DataFrame containing the SMIC data and 'SMIC.extension' is your zip file

# Check if the file exists and rename it to .zip if necessary
if os.path.exists('SMIC.extension'):
  !mv SMIC.extension SMIC.zip # Rename the file to have a .zip extension
  zip_file = 'SMIC.zip'
else:
  zip_file = 'SMIC.zip' # Use the correct filename if it already exists

# Extract the zip file
try:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('/content/SMIC_extracted')  # Extract to a directory named 'SMIC_extracted'
except zipfile.BadZipFile:
    print(f"Error: {zip_file} is not a valid zip file. Please check the file and try again.")
    # Handle the error, e.g., by downloading the file again or exiting the script


# Define source and destination directories (adjust paths as needed)
source_dir = '/content/SMIC_extracted/SMIC_all_cropped/'
dest_dir = './drive/MyDrive/FYP/dataset/mmnet_balanced_SMIC/'
# Initialize counters for each label
copy_counts = {0: 0, 1: 0, 2: 0}

# Iterate through the DataFrame containing the SMIC data
for index, row in df.iterrows():
    subject = 's' + str(row['Subject'])
    filename = row['Filename']
    label = row['Emotion number ']

    # Construct source and destination file paths
    source_image_path = os.path.join(source_dir, subject, filename )
    dest_ep_dir = os.path.join(dest_dir, subject, filename)
    os.makedirs(dest_ep_dir, exist_ok=True)
    dest_ep_path = os.path.join(dest_ep_dir, subject, filename )
    source_folder_path = os.path.join(source_dir, subject, filename)

    if os.path.exists(source_folder_path):
        # Copy images from the source folder to the destination
        for image_file in os.listdir(source_folder_path):
            if image_file.endswith(".jpg"):  # Check if the file is a JPG image
                src_image_path = os.path.join(source_folder_path, image_file)
                dest_image_path = os.path.join(dest_ep_dir, image_file)

                # Check if the source image exists before copying
                if os.path.exists(source_image_path):
                    shutil.copy(source_image_path, dest_image_path)  # Copy the image
                    copy_counts[label] += 1  # Count successful copy
                    print(f"Copied {source_image_path} to {dest_image_path}")
                else:
                    print(f"Source image not found: {src_image_path}")
    else:
        print(f"Source image not found: {source_image_path}")

# Print the number of copied files for each label
print("\nFile copying summary:")
for label, count in copy_counts.items():
    print(f"Label {label}: {count} files copied")

Error: SMIC.zip is not a valid zip file. Please check the file and try again.
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_po_01
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_po_02
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_po_03
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_sur_01
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_sur_02
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_sur_03
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_sur_04
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s20/s20_sur_05
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s14/s14_po_01
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s14/s14_po_02
Source image not found: /content/SMIC_extracted/SMIC_all_cropped/s14/s14_sur_01
Source image not found: /content/SMIC_extracted